##  Тема "SQL. Анализ баз данных"

Анализ баз данных с помощью SQL запросов. В базе — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Необходимо сформулировать ценностное предложение для сервиса.

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
 'pwd': 'Sdf4$2;d-d30pp', # пароль
 'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
 'port': 6432, # порт подключения
 'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

In [2]:
# данные таблицы books
query = '''SELECT * FROM books
           LIMIT 5'''
pd.io.sql.read_sql(query, con = engine) 

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [3]:
# данные таблицы authors
query = '''SELECT * FROM authors
             LIMIT 5'''
pd.io.sql.read_sql(query, con = engine) 

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [4]:
# данные таблицы publishers
query = '''SELECT * FROM publishers
                  LIMIT 5'''
pd.io.sql.read_sql(query, con = engine) 

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [5]:
# данные таблицы ratings
query = '''SELECT * FROM ratings
                  LIMIT 5'''
pd.io.sql.read_sql(query, con = engine) 

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [6]:
# данные таблицы reviews
query = '''SELECT * FROM reviews
                  LIMIT 5'''
pd.io.sql.read_sql(query, con = engine) 

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


#### Задача 1. Сколько книг вышло после 1 января 2000 года. <a id="cell2"></a>

In [7]:
query = '''SELECT 
           COUNT(*) AS cnt
           FROM books
           WHERE publication_date > '2000-01-01'
           '''
pd.io.sql.read_sql(query, con = engine) 

,cnt
0,819


В нашей базе 1000 книг, из них после 1 января 2000, вышло 819 книг.

#### Задача 2. Для каждой книги посчитать количество обзоров и среднюю оценку. <a id="cell2"></a>

In [8]:
query = ''' 
SELECT
t3.book_id,
t3.title,
t1.review_cnt,
t2.average
FROM (SELECT 
      book_id,
      COUNT(review_id) AS review_cnt
      FROM reviews
      GROUP BY 
      book_id)t1
INNER JOIN (
      SELECT 
      book_id,
      AVG(rating) AS average
      FROM ratings
      GROUP BY 
      book_id)t2 ON t2.book_id = t1.book_id
RIGHT JOIN (
      SELECT 
      book_id,
      title
      FROM books)t3 ON t3.book_id = t1.book_id   
ORDER BY t2.average DESC
LIMIT 20
           '''
pd.io.sql.read_sql(query, con = engine)

,book_id,title,review_cnt,average
0,672,The Cat in the Hat and Other Dr. Seuss Favorites,NaN,NaN
1,83,Anne Rice's The Vampire Lestat: A Graphic Novel,NaN,NaN
2,387,Leonardo's Notebooks,NaN,NaN
3,191,Disney's Beauty and the Beast (A Little Golden...,NaN,NaN
4,221,Essential Tales and Poems,NaN,NaN
5,808,The Natural Way to Draw,NaN,NaN
6,642,The Big Bad Wolf (Alex Cross #9),2.0,5.0
7,297,Hard Times,2.0,5.0
8,459,My Name Is Asher Lev,2.0,5.0
9,57,Act of Treason (Mitch Rapp #9),2.0,5.0


Несколько книг не имеют оценок и отзывов. Не всегда оценка 5 гарантирует отличную книгу, т.к. ее мог поставить один пользователь.

#### Задача 3. Определить издательство, которое выпустило наибольшее число книг толще 50 страниц. <a id="cell3"></a>

In [9]:
query = ''' 
SELECT
publishers.publisher_id AS publisher_id,
publishers.publisher AS publisher,
COUNT(publishers.publisher_id) AS publisher_cnt
FROM books
LEFT JOIN publishers ON publishers.publisher_id  = books.publisher_id 
WHERE num_pages > 50
GROUP BY 
publishers.publisher_id
ORDER BY publisher_cnt DESC
LIMIT 5
'''
pd.io.sql.read_sql(query, con = engine) 

,publisher_id,publisher,publisher_cnt
0,212,Penguin Books,42
1,309,Vintage,31
2,116,Grand Central Publishing,25
3,217,Penguin Classics,24
4,35,Bantam,19


В наш топ по выпуску книг попали издания Penguin Books	- 42 книги, Vintage	- 31 книга, Grand Central Publishing - 25 книг.

#### Задача 4. Определить автора с самой высокой средней оценкой книг — учитывать только книги с 50 и более оценками.  <a id="cell4"></a>

In [10]:
query = ''' 
SELECT
author,
AVG(average) AS average_author
FROM
    (SELECT
    *
    FROM
        (
        SELECT
        *
        FROM (
            SELECT
            book_id,
            AVG(rating) AS average,
            COUNT(rating_id) AS rating_cnt
            FROM ratings
            GROUP BY book_id)t
        WHERE t.rating_cnt >= 50)t1
    LEFT JOIN books ON books.book_id = t1.book_id
    LEFT JOIN authors ON authors.author_id  = books.author_id)t2
GROUP BY author
ORDER BY average_author DESC
'''
pd.io.sql.read_sql(query, con = engine) 

,author,average_author
0,J.K. Rowling/Mary GrandPré,4.283844
1,Markus Zusak/Cao Xuân Việt Khương,4.264151
2,J.R.R. Tolkien,4.258446
3,Louisa May Alcott,4.192308
4,Rick Riordan,4.080645
5,William Golding,3.901408
6,J.D. Salinger,3.825581
7,Paulo Coelho/Alan R. Clarke/Özdemir İnce,3.789474
8,William Shakespeare/Paul Werstine/Barbara A. M...,3.787879
9,Dan Brown,3.754540


Авторы с самой высокой средней оценкой книг (учитывая только книги с 50 и более оценками) -это J.K. Rowling/Mary GrandPré, серия книг про Гарри Потера находится в топе,  а у Markus Zusak/Cao Xuân Việt Khương в топе одна книга и надо учитывать данные факторы.

#### Задача 5. Посчитать среднее количество обзоров от пользователей, которые поставили больше 50 оценок.  <a id="cell5"></a>

In [11]:
query = ''' 
SELECT
AVG(t2.review_id_cnt) AS average_review
FROM
    (SELECT
    reviews.username AS username,
    COUNT(review_id) AS review_id_cnt
    FROM
        (SELECT
        ratings.username AS username,
        COUNT(username) AS username_cnt
        FROM books
        LEFT JOIN ratings ON ratings.book_id = books.book_id
        GROUP BY username
        HAVING COUNT(username) > 50)t
    LEFT JOIN reviews ON reviews.username = t.username
    GROUP BY reviews.username)t2

'''
pd.io.sql.read_sql(query, con = engine) 

,average_review
0,24.333333


Самые активные пользователи, которые ставят оценки и пишут обзоры, в среднем написали по 24 обзора. 